In [19]:
import pandas as pd
import os
from typing import List, Sequence, Optional
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from typing import Mapping, Hashable, Optional, Iterable
from plotly.subplots import make_subplots
from statsmodels.formula.api import ols
import re
from backend.cuentas_nacionales.pib import PIBViews
from backend.cuentas_nacionales.pib.pib_constantes import ConstantesPIB as C
from backend.political_terms import tag_politics
from backend.plots import (plot_timeseries_bars, plot_bar_subplots_by_administration, 
                           simple_linegraph, plot_bars, create_pie_chart, 
                           plot_contributions, plot_admin_sector_dumbbell)
from backend.utils import compute_basic_statistics, calculate_component_percentages, validate_percentage_sum, aggregate_columns
from backend.contributions import compute_contributions_pipeline, summary_contributions

In [20]:
g = lambda x: x.map(lambda x:x if type(x) == str else '{:,.2f}'.format(x))

year_on_year_changes = lambda df, q: (df.pct_change(q) * 100).dropna()

In [21]:
# pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [22]:
# colores por presidente:

presidential_colors = {
    "Calderón":   "#9ecae1",  
    "Olsen":      "#fb6a4a",  
    "Rodríguez":  "#fdd0a2",  
    "Pacheco":    "#9467bd",  
    "Arias":      "#74c476",  
    "Chinchilla": "#fdae6b",  
    "Solís":      "#c7c7c7",  
    "Alvarado":   "#e377c2",  
    "Chaves":     "#6baed6",  
}

# <font color = 'red'> 1. DATOS

In [23]:
folder = r'C:\Users\adolj\OneDrive\Documentos\APPS\informed_economist\data\raw'
file_name = 'Variables_PIB_TC.xlsx'
input_path = os.path.join(folder, file_name)
quarterly_data_levels_TC = pd.read_excel(input_path, index_col= 'fecha')

# Datos en millones de colones
quarterly_data_levels_TC.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 139 entries, 1991-03-31 to 2025-09-30
Data columns (total 45 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   PIB_TC                                       139 non-null    float64
 1   PIB_Impuestos_TC                             139 non-null    float64
 2   PIB_Valor_Agregado_TC                        139 non-null    float64
 3   PIB_Agricultura_Silvicultura_Pesca_TC        139 non-null    float64
 4   PIB_Minas_Canteras_TC                        139 non-null    float64
 5   PIB_Manufactura_TC                           139 non-null    float64
 6   PIB_Electricidad_Agua_Saneamiento_TC         139 non-null    float64
 7   PIB_Construccion_TC                          139 non-null    float64
 8   PIB_Comercio_TC                              139 non-null    float64
 9   PIB_Transporte_Almacenamiento_TC             139 non-null

In [24]:
opibv = PIBViews(quarterly_data_levels_TC)

## <font color = 'red'> 1.1 DATOS EN TASAS DE CRECIMIENTO INTERANUAL

### <font color = 'red'> 1.1 TOTAL

In [25]:
# datos en millones de colones
pib = opibv.oferta.get_pib()
# datos en tasas de crecimiento interanuales
pib = year_on_year_changes(pib, q = 4)
# etiqueta de administración
pib = tag_politics(pib)

pib

,PIB_TC,President,Party,Term,Label
fecha,,,,,
1992-03-31,8.218815,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-06-30,9.265592,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-09-30,9.733945,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-12-31,8.975770,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1993-03-31,8.043025,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...
2024-09-30,4.245729,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,4.329637,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,4.113338,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


In [26]:
# datos
df = pib


df.columns = df.columns.str.replace('_TC', '')
col = 'PIB'
statistic = 'mean'
df = df[df['Label'] != 'Calderón']

fig_pib_label = plot_timeseries_bars(
    df=df,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del {col} por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

fig_pib_label.add_annotation(
    text="Adol J. C.",
    xref="paper", yref="paper",
    x=-0.07,  # bien pegado al borde izquierdo
    y=-0.18,  # bien abajo
    showarrow=False,
    font=dict(
        size=12,                    # tamaño pequeño
        color="skyblue",   # opacidad baja (35%)
    ),
    align="left"
)

fig_pib_label.show()

stats_by_admin = compute_basic_statistics(df, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)
display(stats_by_admin[[statistic]].transpose())



Label,Arias,Chaves,Rodríguez,Pacheco,Chinchilla,Olsen,Solís,Alvarado
mean,4.8,4.51,4.34,4.25,4.18,4.12,3.88,2.35


In [27]:
# orden canónico
admin_order = stats_by_admin.index

In [28]:
round(pib[(pib['Label'] == 'Alvarado')].loc[:'2020-03-31']['PIB'].mean(), 2)

np.float64(2.24)

<font color = 'salmon'> ¿En cuáles administraciones la economía costarricense creció más rápido y en cuáles más lento?

Para comparar el ritmo de crecimiento económico por administración, se promediaron las tasas de variación interanual del PIB Tendencia Ciclo correspondientes al período en que cada gobierno estuvo en funciones, desde la administración Figueres Olsen (1994–1998) hasta la administración de Rodrigo Chaves Robles (2022–presente).

Los resultados muestran que las tres administraciones con mayor dinamismo económico han sido:

- Oscar Arias (2006–2010), con un crecimiento promedio de 4.80%,

- Rodrigo Chaves (2022–presente), con 4.51%, y

- Miguel Ángel Rodríguez (1998–2002), con 4.34%.

En contraste, las administraciones con menor ritmo de crecimiento fueron:

- José María Figueres Olsen (1994–1998), con 4.12%,

- Luis Guillermo Solís (2014–2018), con 3.88%, y

- Carlos Alvarado Quesada (2018–2022), con un promedio significativamente menor de 2.35%, el más bajo del periodo analizado.

Un punto relevante es que, si bien el choque económico de la pandemia ocurrió durante el gobierno de Carlos Alvarado, la economía ya mostraba un desempeño débil desde antes del COVID-19: en los trimestres previos al inicio de la crisis sanitaria, el crecimiento promedio apenas alcanzaba 2.24%, revelando una desaceleración que antecedía al impacto pandémico.

### <font color = 'red'> 1.2 OFERTA: ACTIVIDAD ECONÓMICA

In [29]:
# datos en millones de colones
pib_ind = opibv.oferta.get_industrias()
# datos en tasas de crecimiento interanuales
pib_ind = year_on_year_changes(pib_ind, q = 4)
# etiqueta de administración
pib_ind = tag_politics(pib_ind)

display(g(pib_ind.tail(3)))

,PIB_Agricultura_Silvicultura_Pesca_TC,PIB_Minas_Canteras_TC,PIB_Manufactura_TC,PIB_Electricidad_Agua_Saneamiento_TC,PIB_Construccion_TC,PIB_Comercio_TC,PIB_Transporte_Almacenamiento_TC,PIB_Hoteles_Restaurantes_TC,PIB_Informacion_Comunicaciones_TC,PIB_Financieras_Seguros_TC,PIB_Inmobiliario_TC,PIB_Actividades_Profesionales_TC,PIB_Administracion_Publica_TC,PIB_Ense_Salud_Asistencia_Social_TC,PIB_Otras_Actividades_TC,President,Party,Term,Label
fecha,,,,,,,,,,,,,,,,,,,
2025-03-31,-2.93,1.33,8.49,3.69,1.89,2.47,6.82,-0.09,3.91,6.53,4.74,5.75,1.47,2.14,1.76,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-06-30,-2.60,2.71,11.19,5.47,-1.06,2.70,6.22,1.68,3.62,5.90,4.42,5.68,1.27,2.43,1.60,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-09-30,-1.92,3.32,12.80,8.87,-2.76,3.15,5.64,2.66,4.08,5.99,4.11,6.76,0.82,2.49,1.80,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


In [30]:
# datos
df = pib_ind

df.info()

print('\n')

#df.columns = df.columns.str.replace('_TC', '')
col = 'PIB_Agricultura_Silvicultura_Pesca_TC'
statistic = 'mean'
df = df[df['Label'] != 'Calderón']

fig_pib_label = plot_timeseries_bars(
    df=df,
    start = '1994-06-30',
    y_col=col,
    category_col="Label",
    title=f"Costa Rica: Crecimiento Interanual del {col} por Administración",
    x_title="",
    y_title="(%)",
    y_round=2,
    y_suffix="",
    width=850,
    height=420,
    zero_line=False,
    showlegend=True,
    colors=presidential_colors,
    template='plotly_white',
    show_y_grid=True
)

fig_pib_label.show()

stats_by_admin = compute_basic_statistics(df, value_col=col, groupby_col='Label')
stats_by_admin = stats_by_admin.sort_values(by = statistic, ascending = False).round(2)
display(stats_by_admin[[statistic]].transpose())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 135 entries, 1992-03-31 to 2025-09-30
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   PIB_Agricultura_Silvicultura_Pesca_TC  135 non-null    float64
 1   PIB_Minas_Canteras_TC                  135 non-null    float64
 2   PIB_Manufactura_TC                     135 non-null    float64
 3   PIB_Electricidad_Agua_Saneamiento_TC   135 non-null    float64
 4   PIB_Construccion_TC                    135 non-null    float64
 5   PIB_Comercio_TC                        135 non-null    float64
 6   PIB_Transporte_Almacenamiento_TC       135 non-null    float64
 7   PIB_Hoteles_Restaurantes_TC            135 non-null    float64
 8   PIB_Informacion_Comunicaciones_TC      135 non-null    float64
 9   PIB_Financieras_Seguros_TC             135 non-null    float64
 10  PIB_Inmobiliario_TC                    135 non-null    

Label,Olsen,Pacheco,Chinchilla,Solís,Rodríguez,Arias,Chaves,Alvarado
mean,5.85,3.19,2.72,2.19,1.92,1.77,0.75,0.22


In [31]:
df = pib_ind

# 1) Promedio de cada actividad por administración
mean_by_admin = df.drop(['President', 'Party', 'Term'], axis = 1).groupby("Label").mean() 

# 2) Para cada administración: actividad con mayor crecimiento y menor crecimiento
summary = pd.DataFrame({
    "Max_Activity": mean_by_admin.idxmax(axis=1),
    "Max_Growth":   mean_by_admin.max(axis=1),
    "Min_Activity": mean_by_admin.idxmin(axis=1),
    "Min_Growth":   mean_by_admin.min(axis=1),
})

summary = summary.loc[admin_order]

g(round(summary, 2))


,Max_Activity,Max_Growth,Min_Activity,Min_Growth
Label,,,,
Arias,PIB_Financieras_Seguros_TC,14.38,PIB_Manufactura_TC,-0.39
Chaves,PIB_Actividades_Profesionales_TC,7.97,PIB_Administracion_Publica_TC,0.37
Rodríguez,PIB_Informacion_Comunicaciones_TC,19.04,PIB_Comercio_TC,0.69
Pacheco,PIB_Informacion_Comunicaciones_TC,18.42,PIB_Administracion_Publica_TC,1.23
Chinchilla,PIB_Informacion_Comunicaciones_TC,11.46,PIB_Construccion_TC,-2.00
Olsen,PIB_Inmobiliario_TC,12.63,PIB_Transporte_Almacenamiento_TC,-2.77
Solís,PIB_Informacion_Comunicaciones_TC,10.00,PIB_Administracion_Publica_TC,1.14
Alvarado,PIB_Manufactura_TC,6.90,PIB_Construccion_TC,-2.47


In [32]:
SHORT_NAMES = {
    "PIB_Agricultura_Silvicultura_Pesca_TC": "Agro",
    "PIB_Minas_Canteras_TC": "Minas",
    "PIB_Manufactura_TC": "Manuf.",
    "PIB_Electricidad_Agua_Saneamiento_TC": "Energía",
    "PIB_Construccion_TC": "Const.",
    "PIB_Comercio_TC": "Come.",
    "PIB_Transporte_Almacenamiento_TC": "Transp.",
    "PIB_Hoteles_Restaurantes_TC": "Turismo",
    "PIB_Informacion_Comunicaciones_TC": "Info.",
    "PIB_Financieras_Seguros_TC": "Finanzas",
    "PIB_Inmobiliario_TC": "Inmob.",
    "PIB_Actividades_Profesionales_TC": "Prof.",
    "PIB_Administracion_Publica_TC": "Adm. P.",
    "PIB_Ense_Salud_Asistencia_Social_TC": "Educ & Salud",
    "PIB_Otras_Actividades_TC": "Otros Serv.",
}

summary_pretty = summary.copy()
summary_pretty["Max_Activity"] = summary_pretty["Max_Activity"].map(SHORT_NAMES).fillna(summary_pretty["Max_Activity"])
summary_pretty["Min_Activity"] = summary_pretty["Min_Activity"].map(SHORT_NAMES).fillna(summary_pretty["Min_Activity"])

fig = plot_admin_sector_dumbbell(
    summary_pretty,
    title_color = 'gray',
    axis_label_color  = 'gray',
    presidential_colors = presidential_colors,
    admin_order=admin_order,
    x_range=(-7, 22),
)
fig.update_layout(autosize=True)

fig.add_annotation(
    text="Adol J. C.",
    xref="paper", yref="paper",
    x=-0.12,  # bien pegado al borde izquierdo
    y=-0.07,  # bien abajo
    showarrow=False,
    font=dict(
        size=12,                    # tamaño pequeño
        color="skyblue",   # opacidad baja (35%)
    ),
    align="left"
)


fig.show()

<font color = 'salmon'> ¿Cuál actividad económica creció más y cuál menos en cada administración?

Durante la administración de Óscar Arias (2006–2010), el mayor crecimiento provino del sector de Servicios Financieros y Seguros, con un incremento promedio cercano al 14.4%. En contraste, Manufactura registró la menor variación, con una ligera caída del –0.39%.

En la administración de Rodrigo Chaves (2022–presente), la actividad más dinámica ha sido Actividades Profesionales, con un crecimiento del 7.97%, mientras que Administración Pública mostró el menor avance, aunque positivo, con 0.37%.

Durante el gobierno de Miguel Ángel Rodríguez (1998–2002), el principal motor económico fue Información y Comunicaciones, con un notable crecimiento del 19.04%. Por su parte, la actividad con menor dinamismo fue Comercio, con un incremento moderado de 0.69%.

En la administración de Abel Pacheco (2002–2006), también destacó Información y Comunicaciones como la actividad de mayor crecimiento, con 18.42%, mientras que Administración Pública nuevamente figuró como el sector menos dinámico (1.23%).

Bajo la administración de Laura Chinchilla (2010–2014), Información y Comunicaciones lideró con un crecimiento del 11.46%, mientras que Construcción presentó la mayor contracción, con –2.00%.

En el periodo de José María Figueres Olsen (1994–1998), la actividad más dinámica fue Inmobiliario, con un crecimiento del 12.63%, en tanto que Transporte y Almacenamiento experimentó la mayor caída del conjunto, con –2.77%.

Durante el gobierno de Luis Guillermo Solís (2014–2018), Información y Comunicaciones volvió a ser la actividad con mayor crecimiento (10.00%), mientras que Administración Pública registró la menor variación (1.14%).

Finalmente, en la administración de Carlos Alvarado (2018–2022), Manufactura fue la actividad con mejor desempeño, con un crecimiento del 6.90%, mientras que Construcción presentó una contracción significativa de –2.47%.

En conjunto, estos resultados muestran que si bien ciertas actividades —como Información y Comunicaciones— han liderado de forma recurrente en múltiples administraciones, cada período exhibe una dinámica particular respecto a cuál sector actuó como motor principal y cuáles se rezagaron.

### <font color = 'red'> 1.3 OFERTA: SECTORES

In [35]:
# datos en millones de colones
pib_sectores = opibv.oferta.get_sectores()
# datos en tasas de crecimiento interanuales
pib_sectores = year_on_year_changes(pib_sectores, q = 4)
# etiqueta de administración
pib_sectores = tag_politics(pib_sectores)

display(g(pib_sectores))

,Agro_TC,Servicios_TC,Industria_Ampliada_TC,President,Party,Term,Label
fecha,,,,,,,
1992-03-31,5.24,5.08,11.92,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-06-30,8.91,6.13,12.67,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-09-30,7.23,7.04,12.31,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-12-31,2.32,7.22,8.74,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1993-03-31,1.83,6.57,7.15,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...,...,...
2024-09-30,2.62,4.38,4.03,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,-0.51,4.05,5.24,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,-2.93,3.86,6.63,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


In [ ]:
df = pib_sectores

df.columns = df.columns.str.replace('_TC', '')
df.rename(columns={'Industria_Ampliada': 'Industria'}, inplace = True)
df = df.drop(['President', 'Party', 'Term'], axis = 1)

components = ['Agro', 'Servicios', 'Industria']
means = df.groupby("Label")[components].mean()
to_remove = ["Calderón", "Pacheco", "Rodríguez"]
means = means.drop(index=to_remove, errors="ignore")
fig = plot_bar_subplots_by_administration(df = means, variables = components, colors = presidential_colors, title = "Costa Rica: Crecimiento Promedio del PIB por Administración y Sector", y_title = "Crecimiento Interanual (%)")
fig.show()

g(means.loc[['Chaves', 'Alvarado', 'Solís', 'Chinchilla', 'Arias']])




,Agro,Servicios,Industria
Label,,,
Chaves,0.75,4.43,5.52
Alvarado,0.22,1.90,4.00
Solís,2.19,4.58,2.11
Chinchilla,2.72,5.20,1.21
Arias,1.77,6.04,2.35


### <font color = 'red'> 1.4 DEMANDA: COMPONENTES DEL GASTO

In [17]:
# datos en millones de colones
pib_comp_gasto = opibv.demanda.get_categorias_principales()
# datos en tasas de crecimiento interanuales
pib_comp_gasto = year_on_year_changes(pib_comp_gasto, q = 4)
# etiqueta de administración
pib_comp_gasto = tag_politics(pib_comp_gasto)

pib_comp_gasto.drop('PIB_TC', axis = 1, inplace = True)

display(g(pib_comp_gasto.tail(3)))

,PIB_Gasto_Consumo_Final_Hogares_TC,PIB_Gasto_Consumo_Final_Gobierno_General_TC,PIB_Formacion_Bruta_Capital_Fijo_TC,PIB_Exportaciones_Bienes_Servicios_TC,PIB_Importaciones_Bienes_Servicios_TC,President,Party,Term,Label
fecha,,,,,,,,,
2025-03-31,3.08,2.19,6.96,6.97,3.89,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-06-30,2.77,2.28,3.18,8.39,4.91,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-09-30,3.67,2.12,2.24,9.88,7.65,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves


### <font color = 'red'> 1.4 RÉGIMEN DE COMERCIO (RÉGIMEN DEFINITIVO Y ZONAS FRANCAS)

In [18]:
# datos en millones de colones
pib_regimen = opibv.dataframe[['PIB_RegDef_TC', 'PIB_RegEsp_TC']]
# datos en tasas de crecimiento interanuales
pib_regimen = year_on_year_changes(pib_regimen, q = 4)
# etiqueta de administración
pib_regimen = tag_politics(pib_regimen)

display(g(pib_regimen))

,PIB_RegDef_TC,PIB_RegEsp_TC,President,Party,Term,Label
fecha,,,,,,
1992-03-31,7.81,25.63,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-06-30,8.96,24.30,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-09-30,9.50,25.83,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1992-12-31,8.74,27.44,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
1993-03-31,7.81,26.76,Rafael Ángel Calderón Fournier,PUSC,1990–1994,Calderón
...,...,...,...,...,...,...
2024-09-30,3.59,8.20,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2024-12-31,3.53,9.88,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
2025-03-31,3.22,10.04,Rodrigo Chaves Robles,PPSD,2022–2026,Chaves
